In [33]:
%load_ext autoreload
%autoreload 2

from models.clus_gcn_model import ClusGCN, ClusGCNModel

model_wrapper = ClusGCNModel(
        size="100k",
        embedding_dim=64,
        num_layers=5,
        learning_rate=0.01,
        epochs=50,
        device='cpu'
    )

2024-10-31 21:10:22,575 - LGCN model - [INFO]: Using CPU


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
model_wrapper.prepare_training_data()

2024-10-31 21:10:22,843 - LGCN model - [INFO]: Preparing data...
2024-10-31 21:10:23,269 - LGCN model - [INFO]: Data Loaded.


   user  item  rating  timestamp
0   196   242       3  881250949
1   186   302       3  891717742
2    22   377       1  878887116
3   244    51       2  880606923
4   166   346       1  886397596


2024-10-31 21:10:23,754 - LGCN model - [INFO]: Generating negative samples...
2024-10-31 21:10:24,298 - LGCN model - [INFO]: Generating negative samples...
2024-10-31 21:10:24,451 - LGCN model - [INFO]: building graph...
2024-10-31 21:10:24,486 - LGCN model - [INFO]: initilizing model...


adj_norm: tensor(indices=tensor([[   0,    0,    0,  ..., 2622, 2623, 2624],
                       [ 943,  953, 1026,  ...,  650,  650,  650]]),
       values=tensor([0.0151, 0.0094, 0.0099,  ..., 0.0382, 0.0382, 0.0382]),
       size=(2625, 2625), nnz=198114, layout=torch.sparse_coo)


## Generate Initial Encoding for User and Movie

In [35]:
import torch
import os
torch.load("genres_hot.pt").size()

torch.Size([9742, 20])

In [36]:
model_wrapper.user_features

,userid,age,gender,occupation,zipcode
0,1,18-24,M,technician,85711
1,2,50-55,F,other,94043
2,3,18-24,M,writer,32067
3,4,18-24,M,technician,43537
4,5,25-34,F,other,15213
...,...,...,...,...,...
938,939,25-34,F,student,33319
939,940,25-34,M,administrator,02215
940,941,18-24,M,student,97229
941,942,45-49,F,librarian,78209


In [37]:
ages2code = {"Under 18":0, "18-24":1, "25-34":2, "35-44":3, "45-49":4, "50-55":5, "56+":6}
occupations2code = {
        "administrator": 0,
        "artist": 1,
        "doctor": 2,
        "educator": 3,
        "engineer": 4,
        "entertainment": 5,
        "executive": 6,
        "healthcare": 7,
        "homemaker": 8,
        "lawyer": 9,
        "librarian": 10,
        "marketing": 11,
        "none": 12,
        "other": 13,
        "programmer": 14,
        "retired": 15,
        "salesman": 16,
        "scientist": 17,
        "student": 18,
        "technician": 19,
        "writer": 20
    }



In [38]:
model_wrapper.items

,movie_id,title,year,genre_unknown,genre_Action,genre_Adventure,genre_Animation,genre_Children's,genre_Comedy,genre_Crime,...,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,Toy Story,1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye,1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty,1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat,1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn,1997,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey,1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors,1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy,1994,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
# sort the row order of the movies
# the order follows the "index" instead of the movie_id
model_wrapper.items['sorting_index'] = model_wrapper.items['movie_id'].map(model_wrapper.item2idx)
model_wrapper.items = model_wrapper.items.sort_values('sorting_index').drop(columns='sorting_index').reset_index(drop=True)

In [40]:
model_wrapper.items

,movie_id,title,year,genre_unknown,genre_Action,genre_Adventure,genre_Animation,genre_Children's,genre_Comedy,genre_Crime,...,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,242,Kolya,1996,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,302,L.A. Confidential,1997,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,377,Heavyweights,1994,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,51,Legends of the Fall,1994,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,346,Jackie Brown,1997,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1674,Mamma Roma,1962,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1640,"Eighth Day, The",1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,1637,Girls Town,1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1680,1630,"Silence of the Palace, The (Saimt el Qusur)",1994,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
genre_columns = model_wrapper.items.columns[-19:]
genre_columns

Index(['genre_unknown', 'genre_Action', 'genre_Adventure', 'genre_Animation',
       'genre_Children's', 'genre_Comedy', 'genre_Crime', 'genre_Documentary',
       'genre_Drama', 'genre_Fantasy', 'genre_Film-Noir', 'genre_Horror',
       'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Sci-Fi',
       'genre_Thriller', 'genre_War', 'genre_Western'],
      dtype='object')

In [42]:
genre_values = model_wrapper.items[genre_columns].values
genre_tensor = torch.tensor(genre_values, dtype=torch.float)

In [44]:
print(genre_tensor.shape)
torch.save(genre_tensor, 'genre_hot_embeddings.pt')

torch.Size([1682, 19])


In [47]:
model_wrapper.user_features

,userid,age,gender,occupation,zipcode
0,1,18-24,M,technician,85711
1,2,50-55,F,other,94043
2,3,18-24,M,writer,32067
3,4,18-24,M,technician,43537
4,5,25-34,F,other,15213
...,...,...,...,...,...
938,939,25-34,F,student,33319
939,940,25-34,M,administrator,02215
940,941,18-24,M,student,97229
941,942,45-49,F,librarian,78209


In [48]:
# Users
model_wrapper.user_features['sorting_index'] = model_wrapper.user_features['userid'].map(model_wrapper.user2idx)
model_wrapper.user_features = model_wrapper.user_features.sort_values('sorting_index').drop(columns='sorting_index').reset_index(drop=True)

# Generate one-hot encoding for age
age_encoding = torch.zeros(len(model_wrapper.user_features), len(ages2code))
for i, age_range in enumerate(model_wrapper.user_features['age']):
    age_encoding[i][ages2code[age_range]] = 1

# Generate one-hot encoding for occupation
occupation_encoding = torch.zeros(len(model_wrapper.user_features), len(occupations2code))
for i, occupation in enumerate(model_wrapper.user_features['occupation']):
    occupation_encoding[i][occupations2code[occupation]] = 1

# Save the encodings to .pt files
torch.save(age_encoding, 'user_age_hot_embedding.pt')
torch.save(occupation_encoding, 'user_occupation_hot_embedding.pt')

In [49]:
age_encoding.size(), occupation_encoding.size()

(torch.Size([943, 7]), torch.Size([943, 21]))

In [51]:
grouped = model_wrapper.ratings.groupby('user_idx')

for user, group in grouped:
    #print(user)
    #print(group)
    test_sample = group.sample(n=1, random_state=42)
    train_sample = group.drop(test_sample.index)
    print(test_sample)
    print(train_sample)
    break
    # if len(group) < 2:
    #     train_list.append(group)
    # else:
    #     test_sample = group.sample(n=1, random_state=42)
    #     train_sample = group.drop(test_sample.index)
    #     train_list.append(train_sample)
    #     test_list.append(test_sample)

0
       user  item  rating  timestamp  user_idx  item_idx
0       196   242       3  881250949         0         0
940     196   393       4  881251863         0       528
1133    196   381       4  881251728         0       377
1812    196   251       3  881251274         0       522
1896    196   655       5  881251793         0       431
2374    196    67       5  881252017         0       834
6910    196   306       4  881251021         0       380
7517    196   238       4  881251820         0       329
7842    196   663       5  881251911         0       550
10017   196   111       4  881251793         0        83
10254   196   580       2  881252056         0       632
10981   196    25       4  881251955         0        86
13733   196   286       5  881250949         0       289
14606   196    94       3  881252172         0       363
16834   196   692       5  881252017         0       438
17102   196     8       5  881251753         0       389
17830   196   428       4  88